In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
30172,2024-12-07,Sérvia Primeira Liga,16:00,Cacak 94,Zlatibor,1.89,1.78,161.5,1.88,1.78,-1.5,2.03,1.67,QVc8HCdn,0.529101,0.561798,0.531915,0.561798,0.090898,111.608,16.209357,0.145235,1.2,1.643168,1.369306,133.94,1.398,0.234243,0.167556,16.0,141.566,38.093091,0.269084,1.8,1.643168,0.912871,143.51,1.646,0.593490,0.360565,48.0,74,113,1.81,1.27,107.290,130.936,0.042388,0.038640,0.137599,-2.37,-0.474,-1.877637,0.681859,0.4,-0.281859,-2.42,-0.484,-1.611570,0.576284,0.6,0.023716
30173,2024-12-07,Rússia Superliga,06:00,Chelyabinsk,Novosibirsk,2.76,1.41,146.5,1.85,1.85,3.5,2.00,1.72,U9ZPAmUm,0.362319,0.709220,0.540541,0.540541,0.071539,159.084,77.790870,0.488992,0.6,1.341641,2.236068,279.37,2.208,0.920989,0.417114,-46.0,165.024,71.622242,0.434011,2.4,1.341641,0.559017,77.88,2.186,0.934789,0.427625,-5.0,91,66,3.07,1.18,115.330,158.140,0.457839,0.000000,0.106446,0.28,0.056,31.428571,0.613274,0.8,0.186726,1.79,0.358,1.145251,0.506516,0.5,-0.006516
30174,2024-12-07,Rússia Superliga,10:00,Tambov,Bars Rostov,1.64,2.06,151.5,1.83,1.83,-3.5,1.98,1.70,xCfoIVyE,0.609756,0.485437,0.546448,0.546448,0.095193,379.912,186.730973,0.491511,1.2,1.643168,1.369306,594.52,4.812,2.162330,0.449362,-7.0,234.352,201.776674,0.860998,1.2,1.643168,1.369306,124.64,3.112,2.575416,0.827576,-4.0,89,76,6.68,1.64,182.168,312.582,0.160532,0.000000,0.107603,-1.64,-0.328,-1.951220,0.509256,0.4,-0.109256,-1.63,-0.326,-3.251534,0.272210,0.3,0.027790
30175,2024-12-07,Sérvia Primeira Liga,12:00,Dynamic,OKK Beograd,1.29,3.15,181.5,1.82,1.84,-8.5,1.96,1.72,fBkPDUeB,0.775194,0.317460,0.549451,0.543478,0.092654,168.436,53.560856,0.317989,2.4,1.341641,0.559017,178.64,1.728,0.512708,0.296706,48.0,216.294,88.233470,0.407933,1.2,1.643168,1.369306,120.65,2.678,1.146133,0.427981,-25.0,116,95,1.54,1.27,169.248,306.584,0.592441,0.007728,0.092231,3.63,0.726,0.399449,0.613658,0.8,0.186342,5.72,1.144,1.879371,0.348691,0.4,0.051309
30176,2024-12-07,Sérvia Primeira Liga,14:00,Joker,Metalac,1.06,6.56,167.5,1.81,1.85,-15.5,2.03,1.67,z1ZkaVtI,0.943396,0.152439,0.552486,0.540541,0.095835,131.288,28.089637,0.213954,1.8,1.643168,0.912871,173.84,1.494,0.402095,0.269140,26.0,233.076,55.757727,0.239226,0.6,1.341641,2.236068,148.23,3.234,0.647248,0.200139,-63.0,82,61,2.12,2.43,118.290,275.814,1.020758,0.015456,0.137599,1.33,0.266,0.225564,0.726128,0.7,-0.026128,-1.36,-0.272,-20.441176,0.338652,0.2,-0.138652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30552,2024-12-07,Europa Liga Bnxt,16:30,Spotter Leuven,Belfius Mons,1.31,3.05,156.5,1.80,1.86,-8.5,2.01,1.68,S4Bt8Tuh,0.763359,0.327869,0.555556,0.537634,0.091228,125.832,35.904650,0.285338,1.2,1.643168,1.369306,100.74,1.674,0.546196,0.326282,15.0,243.320,203.795280,0.837561,1.2,1.643168,1.369306,604.45,3.306,2.648534,0.801129,-27.0,73,77,1.38,7.85,98.810,260.008,0.564388,0.023184,0.126474,0.91,0.182,1.703297,0.765341,0.9,0.134659,-1.64,-0.328,-6.250000,0.508818,0.6,0.091182
30553,2024-12-07,Europa Liga Bnxt,16:30,Oostende,Den Helder,1.02,12.50,161.5,1.85,1.82,-26.5,1.97,1.71,0I9l67A4,0.980392,0.080000,0.540541,0.549451,0.060392,98.312,8.171396

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
30193,15:30,Sérvia 1. Zls Feminino,Sloga Pozega F,Crvena zvezda F,1.82,Back Home
30202,12:00,Turquia Superliga,Buyukcekmece,Mersin SK,1.62,Back Home
30203,09:30,Turquia Superliga,Merkezefendi,Petkim Spor,2.82,Back Home
30213,10:30,Suécia Liga De Basquete,Lund F,Alvik F,2.48,Back Home
30219,18:00,Portugal Lfb Feminina,Sanjoanense F,Galitos F,1.64,Back Home
30266,14:00,Eua Ncaa,Furman,Princeton,2.05,Back Home
30280,19:00,Eua Ncaa,Pepperdine,Grambling St.,1.77,Back Home
30282,18:30,Eua Ncaa,Texas A&M-CC,Stephen F. Austin,1.58,Back Home
30283,18:30,Eua Ncaa,Houston Christian,Northwestern St.,1.95,Back Home
30286,18:00,Eua Ncaa,Sacramento State,Nebraska O.,1.91,Back Home
